In [1]:
import os, warnings, sys, yaml
from stream2segment.process.db import get_session
from stream2segment.io.db.models import Station, Segment, concat, Channel
from stream2segment.io.utils import loads_inv
# warnings.simplefilter('always')
_fle = os.path.abspath(os.path.join('.', 'jnconfig.yaml'))
assert os.path.isfile(_fle)
with open(_fle, "r") as _:
    jnconfig = yaml.safe_load(_)

# dbpath_old = jnconfig['dbpath_old']
dbpath = jnconfig['dbpath_new']

import sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from sod.core.dataset import open_dataset

In [2]:
import pandas as pd
dataframe = pd.read_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/globalset.hdf')
dataframe_old = dataframe.copy()
# dataframe = open_dataset('globalset', normalize=False)

In [17]:
# dtmp = dataframe.copy()
# dataframe = dtmp
dataframe.columns

Index(['Segment.db.id', 'dataset_id', 'psd@0.05sec', 'psd@0.1sec',
       'psd@0.2sec', 'psd@0.5sec', 'psd@1sec', 'psd@2sec', 'psd@3sec',
       'psd@5sec', 'psd@9sec', 'outlier', 'subclass', 'window_type',
       'amplitude_ratio', 'event_id', 'station_id', 'event_time',
       'channel_code', 'magnitude', 'distance_km'],
      dtype='object')

In [6]:
import pandas as pd
pd.unique(dataframe.dataset_id)

In [8]:
from IPython.display import clear_output  # https://stackoverflow.com/a/24818304
import time
from datetime import timedelta

EUROPE_DATASET_ID = 1
ME_DATASET_ID = 2

DATASET_ID = EUROPE_DATASET_ID

sess = get_session(dbpath)
# dataframe['location_code'] = ''
qry = sess.query(Segment.id, Channel.location).join(Segment.channel, Segment.station).filter(
    (Channel.location != '') & (Station.id.in_(pd.unique(dataframe[dataframe.dataset_id == DATASET_ID].station_id).tolist()))
).all()


total = len(qry)
print(str(total) + ' non-empty locations found for DATASET %d' % DATASET_ID)
print('Query done, setting locations')

ID = 'Segment.db.id'  #'globalset.id'
LOC = 'location_code'
if LOC not in dataframe.columns:
    dataframe[LOC] = ''

df2 = pd.DataFrame(qry, columns=[ID, LOC])
print('Found on database these distinct locations ' + str(pd.unique(df2[LOC]).tolist()))
print('Merging')
dataframe = dataframe.merge(df2, how='left', on=[ID], validate='m:1')
print(dataframe.columns)

tobeset = dataframe[ID].isin(df2[ID]) & (dataframe.dataset_id == DATASET_ID)
dataframe.loc[tobeset, LOC] = dataframe[LOC + '_y']
dataframe.loc[~tobeset, LOC] = dataframe[LOC + '_x']
dataframe.drop([LOC+'_x', LOC+'_y'], axis=1, inplace=True)

assert len(dataframe) == len(dataframe_old)
print('New locations: %s' % str(pd.unique(dataframe[LOC])))
print('Checking equality')
assert (dataframe[list(_ for _ in dataframe.columns if _ != LOC)].equals(
    dataframe_old[list(_ for _ in dataframe_old.columns if _ != LOC)]))


# now save files:
# dataframe.to_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/allset.hdf',
#                 format='table', mode='w', key='allset')




214246 non-empty locations found for DATASET 1
Query done, setting locations
Found on database these distinct locations ['00', '01']
Merging
Index(['Segment.db.id', 'dataset_id', 'psd@0.05sec', 'psd@0.1sec',
       'psd@0.2sec', 'psd@0.5sec', 'psd@1sec', 'psd@2sec', 'psd@3sec',
       'psd@5sec', 'psd@9sec', 'outlier', 'subclass', 'window_type',
       'amplitude_ratio', 'event_id', 'station_id', 'event_time',
       'channel_code', 'magnitude', 'distance_km', 'location_code_x',
       'location_code_y'],
      dtype='object')
New locations: ['01' '00' '']
Checking equality


AssertionError: 